In [988]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import torch
from torch import nn
from torch.autograd import Variable
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

In [989]:
def str2ts(x):
    time_string = x + ' GMT-0500'
    dt = datetime.datetime.strptime(time_string, '%Y-%m-%d %H:%M:%S GMT%z')
    ts = int(dt.timestamp())
    return ts

In [1005]:
dt = pd.read_csv('./ts_2k.csv',usecols=[0,1,2,3,4])
ts = dt['date'].apply(str2ts)
dt.insert(1, 'ts', ts)

In [1006]:
dt.head(10)

,date,ts,open,high,low,close
0,2022-02-25 15:59:00,1645822740,809.61,809.99,809.20,809.87
1,2022-02-25 15:58:00,1645822680,808.52,809.94,808.29,809.70
2,2022-02-25 15:57:00,1645822620,809.30,809.50,807.24,808.49
3,2022-02-25 15:56:00,1645822560,810.27,810.51,809.11,809.22
4,2022-02-25 15:55:00,1645822500,809.41,810.64,809.00,810.42
5,2022-02-25 15:54:00,1645822440,809.66,810.04,808.81,809.38
6,2022-02-25 15:53:00,1645822380,807.96,809.68,807.91,809.50
7,2022-02-25 15:52:00,1645822320,807.41,808.07,806.71,808.00
8,2022-02-25 15:51:00,1645822260,806.21,807.67,806.11,807.42
9,2022-02-25 15:50:00,1645822200,805.87,806.74,804.51,806.44


In [925]:
def create_dataset(dt, step):
    data_X = []
    data_Y = []
    debug_all = []
    for i in range(len(dt)- step):
        dt_continue = True
        line = '1'
        data_x = [[1.0],]
        data_y = 0
        for j in range(i+step-1, i-1, -1):
            #print(j)
            if dt.loc[j]['ts']-dt.loc[j+1]['ts'] != 60:
                dt_continue = False
                break
            
            value = dt.loc[j]['close']/dt.loc[j+1]['close']
            line += "\t{}".format(value)
            if j==i:
                data_y = value
            else:
                data_x.append([value])
        if not dt_continue:
            #print(i)
            #return
            continue
        data_X.append(data_x)
        data_Y.append(data_y)
        debug_all.append(line)
        if(i%1000 == 0):
            print(line)
            print("==========={}".format(i))
    return debug_all, np.array(data_X, dtype=np.float32), np.array(data_Y, dtype=np.long)

In [841]:
debug_all, data_X, data_Y = create_dataset(dt=dt, step=10)

1	1.0012788517649398	1.0012152174991318	1.0007183374204256	1.0018564356435644	0.9998517603458925	1.0012849341471248	0.9985192862959947	0.9990978967400707	1.0014966171504904	1.0002099543040632
===========0


In [1007]:
def create_dataset_all(dt, step):
    data_X = []
    data_Y = []
    data_time_X = []
    data_time_Y = []
    for i in range(len(dt)-1, step-1, -1):
        dt_continue = True
        #print("i=",i)
        data_x = []
        data_y = None
        data_time_x = []
        data_time_y = None
        for j in range(i, i-step-1, -1):
            #print("-----j=",j)
            if j == i:   #序列中的第一个
                a = dt.loc[j]['open']
                one = [(dt.loc[j]['high']-a)*10.0/a, (dt.loc[j]['low']-a)*10.0/a, (dt.loc[j]['close']-a)*10.0/a] 
                #one = [dt.loc[j]['high']/dt.loc[j]['open'], dt.loc[j]['low']/dt.loc[j]['open'], dt.loc[j]['close']/dt.loc[j]['open']] 

                data_x.append(one)
                data_time_x.append([dt.loc[j]['date']])
            else:
                if dt.loc[j]['ts']-dt.loc[j+1]['ts'] != 60: #判断是否是连续的
                    dt_continue = False
                    break
            
                if j == i-step:  #要预测的
                    data_y = dt.loc[j]['close']/dt.loc[j+1]['close']
                    #data_y = 1 if data_y>1.002 else 0
                    data_y = 1 if data_y>1 else 0
                    data_time_y = dt.loc[j]['date']         
                else:
                    a = dt.loc[j+1]['close']
                    one = [(dt.loc[j]['high']-a)*10.0/a, (dt.loc[j]['low']-a)*10.0/a, (dt.loc[j]['close']-a)*10.0/a]
                    #one = [dt.loc[j]['high']/dt.loc[j+1]['close'], dt.loc[j]['low']/dt.loc[j+1]['close'], dt.loc[j]['close']/dt.loc[j+1]['close']]
                    data_x.append(one) #序列中不是第一个
                    data_time_x.append([dt.loc[j]['date']])
        
        if dt_continue:
            data_X.append(data_x)
            data_Y.append(data_y)
            data_time_X.append(data_time_x)
            data_time_Y.append(data_time_y)
        
        if(i%1000 == 0):
            print("==========={}".format(i))
                
            
    #return data_time, np.array(data_X, dtype=np.float32), np.array(data_Y, dtype=np.long)
    return data_X, data_Y, data_time_X, data_time_Y

In [1008]:
data_X, data_Y, data_time_X, data_time_Y = create_dataset_all(dt=dt, step=20)

===========1000


In [1009]:
print(len(data_Y))
print(sum(data_Y))
i=444
data_X[i], data_Y[i], data_time_X[i], data_time_Y[i]

1879
910


([[0.01549892335723179, -0.013132675516433752, 0.009228366579115174],
  [0.023167848699764022, -0.003073286052009349, 0.014302600472813669],
  [0.011803448967788387, -0.010387035091653727, -0.009560793663909292],
  [0.014177693761815283, -0.001063327032136482, 0.0087429111531192],
  [0.03198998984819553, -0.004603725476308359, 0.025969733456099882],
  [0.0025902465443746153, -0.01836720276920973, 0.0007064308757381664],
  [0.030374381916647528, -0.010595714622085911, 0.026253826230280414],
  [0.027828986766553605, -0.0041097659781832805, 0.021840470626915603],
  [-0.0023433197811344652, -0.05073287326155012, -0.032103481001534984],
  [0.00211578019394593, -0.0307963561563327, -0.026212165736115407],
  [0.026281054070617288, -0.00011785226040624741, 0.0207419978315183],
  [0.018111680858071192, -0.0027049912969847366, -0.0007056499035605381],
  [0.013173061090070859, -0.007997929947543737, 0.00917409611629899],
  [0.030552291421856906, -0.0038777908343130543, 0.012573443008226205],
  [0

In [1010]:
data_X = np.array(data_X, dtype=np.float32)
data_Y = np.array(data_Y, dtype=np.long)
per = np.random.permutation(data_X.shape[0])    #打乱后的行号
#print(per)
data_X = data_X[per, :, :]    #获取打乱后的训练数据
data_Y = data_Y[per]
#print(debug_all[1000])
#print(data_X)
#print(data_X.shape)
print(len(data_Y))
print(len(data_X))
print(data_X.shape)
data_X[444], data_Y[444]

1879
1879
(1879, 20, 3)


(array([[ 0.02659033, -0.00127226,  0.02519084],
        [ 0.0112947 , -0.01027945, -0.00862966],
        [ 0.        , -0.01333672, -0.00558872],
        [ 0.01385253, -0.00025417,  0.00965864],
        [ 0.00355501, -0.0245042 , -0.01980651],
        [-0.0007633 , -0.03383965, -0.03231306],
        [ 0.0061262 , -0.00957219, -0.00957219],
        [ 0.00613207, -0.00498231, -0.00127751],
        [ 0.00677169, -0.01354338,  0.00268312],
        [ 0.01098508, -0.00983548, -0.00012773],
        [ 0.01315672,  0.00012774,  0.00625902],
        [ 0.00523386, -0.00497855,  0.00140421],
        [-0.00255275, -0.0140401 , -0.00663714],
        [ 0.00178811, -0.00830194, -0.00408711],
        [ 0.02184941, -0.01034972,  0.01520514],
        [ 0.01160981, -0.01301319, -0.01160981],
        [ 0.00242684, -0.02375752, -0.02362979],
        [ 0.004097  , -0.0057614 , -0.00089622],
        [ 0.01357252, -0.00588996,  0.00166455],
        [ 0.00755326, -0.00985764,  0.00204834]], dtype=float32),
 1)

In [1011]:
train_size = int(len(data_X) * 0.8)
train_X = data_X[:train_size]
train_Y = data_Y[:train_size]
test_X = data_X[train_size:]
test_Y = data_Y[train_size:]

#train_Y = train_Y.astype(np.long)
#test_Y = test_Y.astype(np.long)

train_X = train_X.reshape(-1,20,3)
train_Y = train_Y.reshape(-1, )
test_X = test_X.reshape(-1,20,3)
test_Y = test_Y.reshape(-1, )



In [1012]:
print(train_X.shape)
print(train_Y.shape)

#train_X
train_X[444]

(1503, 20, 3)
(1503,)


array([[ 0.02659033, -0.00127226,  0.02519084],
       [ 0.0112947 , -0.01027945, -0.00862966],
       [ 0.        , -0.01333672, -0.00558872],
       [ 0.01385253, -0.00025417,  0.00965864],
       [ 0.00355501, -0.0245042 , -0.01980651],
       [-0.0007633 , -0.03383965, -0.03231306],
       [ 0.0061262 , -0.00957219, -0.00957219],
       [ 0.00613207, -0.00498231, -0.00127751],
       [ 0.00677169, -0.01354338,  0.00268312],
       [ 0.01098508, -0.00983548, -0.00012773],
       [ 0.01315672,  0.00012774,  0.00625902],
       [ 0.00523386, -0.00497855,  0.00140421],
       [-0.00255275, -0.0140401 , -0.00663714],
       [ 0.00178811, -0.00830194, -0.00408711],
       [ 0.02184941, -0.01034972,  0.01520514],
       [ 0.01160981, -0.01301319, -0.01160981],
       [ 0.00242684, -0.02375752, -0.02362979],
       [ 0.004097  , -0.0057614 , -0.00089622],
       [ 0.01357252, -0.00588996,  0.00166455],
       [ 0.00755326, -0.00985764,  0.00204834]], dtype=float32)

In [998]:
for i in range(len(test_Y)):
    #if test_X[i][1][0] >=1:
    if test_Y[i] >=1:
        test_Y[i] = 1
    else:
        test_Y[i] = 0
    
for i in range(len(train_Y)):
    #if train_X[i][1][0] >=1:
    if train_Y[i] >=1:
        train_Y[i] = 1
    else:
        train_Y[i] = 0
        
train_x = torch.from_numpy(train_X)
train_y = torch.from_numpy(train_Y)
test_x = torch.from_numpy(test_X)
test_y = torch.from_numpy(test_Y)

In [999]:
len(train_Y),train_Y

(7583, array([1, 1, 1, ..., 0, 0, 0]))

In [1000]:
class TrainDataSet(Dataset):
    def __init__(self,x,y):
        self.x = x
        self.y = y
        return
    
    def __getitem__(self,index):
        return self.x[index], self.y[index]
    
    def __len__(self):
        return len(self.x)

dt_train = TrainDataSet(train_X, train_Y)
#dt_test = TrainDataSet(test_X, test_Y)
train_loader = DataLoader(dt_train, batch_size = 4096,shuffle = True, drop_last=True)

In [1001]:
class lstm(nn.Module):
    def __init__(self,input_size=2,hidden_size=4,output_size=1,num_layer=2):
        super(lstm,self).__init__()
        self.layer1 = nn.LSTM(input_size,hidden_size,num_layer,batch_first=True)
        self.layer2 = nn.Linear(hidden_size,output_size)
        #input_size: embedding的维度，比如一个字由128维向量描述，则该值是128
        # hidden_size：lstm内部隐含层的维度
        # output_size：lstm输出向量的维度，比如一句话经过lstm变成一个64维的向量
        # lstm参数中并不定义序列的长度，就是不定义由几个lstm_cell串联
    
    def forward(self,x):
        # x的结构：[batch * seq_length * input_size]
        # seq_length：序列长度，比如要用前5个单词预测第6个，那么seq_length=5
        x,_ = self.layer1(x)
        #print("x.shape={}".format(x.shape))
        #b,s,h = x.size()
        x = x[:,-1,:]
        x = self.layer2(x)
        #print("before:",x)
        x = F.softmax(x,dim=1)
        #print("after:",x)
        #print("x.shape={}".format(x.shape))

        return x



model = lstm(3, 8, 2,2)

In [1003]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
#optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

def cal_acc(pred_y, true_y, threshold=0.5):
    #print("pred_y:",pred_y)
    #pre = torch.where(pred_y>threshold, 1, 0)
    pre = pred_y.clone()
    for i in range(len(pre)):
        if pre[i][1] >= threshold:
            pre[i][1] = 1
            pre[i][0] = 0
        else:
            pre[i][0] = 1
            pre[i][1] = 0
        
    #print("pre:", pre)
    #acc_all = sum(pre.indices == true_y)/len(true_y)
    TP = 0  # 1-->1
    FP = 0  # 0-->1
    FN = 0  # 1-->0
    TN = 0  # 1-->1
    ALL = 0
    for i in range(len(true_y)):
        #print(true_y[i], pre[i])
        if true_y[i] == 1 and pre[i][1] == 1:
            TP +=1
        elif true_y[i] == 0 and pre[i][1] == 1:
            FP += 1
        elif true_y[i] == 1 and pre[i][0] == 1:
            FN += 1
        elif true_y[i] == 0 and pre[i][0] == 1:
            TN += 1
        ALL += 1
    
    print("ALL:{}, TP:{}, FP:{}, FN:{}, TN:{}".format(ALL, TP, FP, FN, TN))
    
    precision = TP*1.0/(TP+FP+0.000000001)
    recall = TP*1.0/(TP+FN+0.000000001)
    print("threshold={}  precision={}  recall={}".format(threshold, precision, recall))

def eval(model, data_x, data_y):
    model = model.eval()
    var_data = Variable(data_x)
    #print(var_data.shape)
    pred_test = model(var_data) # 测试集的预测结果
    #print("pred_test.shape:{}".format(pred_test.shape))
    #print("data_y".shape)
    prediction = torch.max(pred_test, 1)
    #print(pred_test)
    #print(prediction.indices.shape)
    #print(data_y.shape)
    #print(pred_test)
    #accuracy = sum(prediction.indices == data_y)/len(data_y)
    #print("acc:{}".format(accuracy))
    
    cal_acc(pred_test, data_y, 0.5)
    cal_acc(pred_test, data_y, 0.6)
    cal_acc(pred_test, data_y, 0.7)
    cal_acc(pred_test, data_y, 0.8)
    cal_acc(pred_test, data_y, 0.9)
    print("------------------------")


    
for epoch in range(0):
    for i,data in enumerate(train_loader):
        var_x, var_y = data
        out = model(var_x)
        loss = criterion(out, var_y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch + 1) % 100 == 0: # 每 100 次输出结果
        print('Epoch: {}, Loss: {:.5f}'.format(epoch + 1, loss.data))
        eval(model, test_x, test_y)
        
    
for e in range(500000):
    var_x = Variable(train_x)
    var_y = Variable(train_y)
    #print("var_x.shape:", var_x.shape)
    #print("var_y.shape:", var_y.shape)
    #print("var_x:", var_x)
    #print("var_y:", var_y)
    # 前向传播
    out = model(var_x)
    #print(var_x.shape)
    #print(out.shape)
    #print(out.dtype)
    #print(var_y.shape)
    #print(var_y.dtype)
    #print(out)
    #print(var_y)
    loss = criterion(out, var_y)
    # 反向传播
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    #print(e)
    #print(loss.data)
    
    if (e + 1) % 500 == 0: # 每 100 次输出结果
        print('Epoch: {}, Loss: {:.5f}'.format(e + 1, loss.data))
        eval(model, test_x, test_y)

Epoch: 500, Loss: 0.66522
ALL:1896, TP:392, FP:403, FN:534, TN:567
threshold=0.5  precision=0.4930817610056691  recall=0.4233261339088301
ALL:1896, TP:271, FP:255, FN:655, TN:715
threshold=0.6  precision=0.5152091254743058  recall=0.29265658747268614
ALL:1896, TP:190, FP:166, FN:736, TN:804
threshold=0.7  precision=0.5337078651670402  recall=0.20518358531295336
ALL:1896, TP:115, FP:97, FN:811, TN:873
threshold=0.8  precision=0.5424528301861206  recall=0.1241900647946823
ALL:1896, TP:48, FP:38, FN:878, TN:932
threshold=0.9  precision=0.5581395348772309  recall=0.051835853131693484
------------------------


KeyboardInterrupt: 